In [8]:
import os 
import shutil
import random

import torch
from transformers import AutoModelForObjectDetection, TrainingArguments, Trainer, YolosImageProcessor
from datasets import load_dataset
from PIL import Image, ImageDraw, ImageFont
from datasets import Dataset
import numpy as np
import gc
from sklearn.metrics import precision_recall_fscore_support
from torchmetrics.detection.mean_ap import MeanAveragePrecision


import configparser
import torch
import gc

import matplotlib.pyplot as plt


In [9]:
# config = configparser.ConfigParser()
# config.read("config.ini")
# AUTH_TOKEN = config["auth"]["token"]

BASE_DIR = os.getcwd()
IMAGE_PROCESSOR_GLOBAL = YolosImageProcessor()


In [ ]:
def clear_directory(directory):
    """
    Löscht den gesamten Inhalt eines Verzeichnisses, ohne das Verzeichnis selbst zu entfernen.

    Args:
        directory (str): Pfad zum Verzeichnis.
    """
    if os.path.exists(directory):
        for item in os.listdir(directory):
            item_path = os.path.join(directory, item)
            if os.path.isfile(item_path) or os.path.islink(item_path):
                os.unlink(item_path)  
            elif os.path.isdir(item_path):
                shutil.rmtree(item_path)  


def split_dataset(images_dir, labels_dir, output_dir, test_ratio=0.2, seed=42):
    """
    Teilt einen Datensatz (Bilder + Labels) in Training, Validierung und Test auf.

    Args:
        images_dir (str): Pfad zum Ordner mit den Bildern.
        labels_dir (str): Pfad zum Ordner mit den Labels.
        output_dir (str): Pfad zum Ordner, in dem die aufgeteilten Daten gespeichert werden sollen.
        val_ratio (float): Verhältnis der Validierungsdaten (zwischen 0 und 1). Standard: 0.1.
        test_ratio (float): Verhältnis der Testdaten (zwischen 0 und 1). Standard: 0.1.
        seed (int): Zufallssaat für Reproduzierbarkeit. Standard: 42.

    Returns:
        None
    """


    # Unterordner-Pfade
    train_images_dir = os.path.join(output_dir, "train/images")
    train_labels_dir = os.path.join(output_dir, "train/labels")
    test_images_dir = os.path.join(output_dir, "test/images")
    test_labels_dir = os.path.join(output_dir, "test/labels")
    
    # Ordner neu erstellen
    os.makedirs(train_images_dir, exist_ok=True)
    os.makedirs(train_labels_dir, exist_ok=True)
    os.makedirs(test_images_dir, exist_ok=True)
    os.makedirs(test_labels_dir, exist_ok=True)

    for subdir in [train_images_dir, train_labels_dir, test_images_dir, test_labels_dir]:
        clear_directory(subdir)
        os.makedirs(subdir, exist_ok=True)

    # Liste aller Bilder
    image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png'))]
    
    # Shuffle und Split
    random.seed(seed)
    random.shuffle(image_files)
    
    test_split = int(len(image_files) * test_ratio)
    
    test_files = image_files[:test_split]
    train_files = image_files[test_split:]

    # Dateien kopieren
    def copy_files(file_list, dest_images_dir, dest_labels_dir):
        for file in file_list:
            shutil.copy(os.path.join(images_dir, file), os.path.join(dest_images_dir, file))
            label_file = file.rsplit('.', 1)[0] + '.txt'
            if os.path.exists(os.path.join(labels_dir, label_file)):
                shutil.copy(os.path.join(labels_dir, label_file), os.path.join(dest_labels_dir, label_file))

    copy_files(train_files, train_images_dir, train_labels_dir)
    copy_files(test_files, test_images_dir, test_labels_dir)

    print(f"Train/Validation/Test-Split abgeschlossen! Daten in '{output_dir}' gespeichert.")
    print(f"Train: {len(train_files)} | Test: {len(test_files)}")

# Beispielaufruf
yolo_images_dir = os.path.join(BASE_DIR, "Data", "Kugellager_Data", "YOLO_data", "yolo_images_dump")
yolo_labels_dir = os.path.join(BASE_DIR, "Data", "Kugellager_Data", "YOLO_data", "yolo_labels_dump")
yolo_output_dir = os.path.join(BASE_DIR, "Data", "Kugellager_Data", "YOLO_data")

split_dataset(images_dir=yolo_images_dir,
              labels_dir=yolo_labels_dir,
              output_dir=yolo_output_dir,
              test_ratio=0.2,
              seed=42)


In [4]:
# split_dataset(images_dir=os.path.join(BASE_DIR, "Data", "Oberflächen_Data", "YOLO_data", "test", "images"),
#               labels_dir=os.path.join(BASE_DIR, "Data", "Oberflächen_Data", "YOLO_data", "test", "labels"),
#               output_dir="halbe_data_oberfläche",
#               test_ratio=0.5)

In [5]:
def convert_yolo_to_custom_format(images_dir, labels_dir, categories):
    """
    Converts YOLO annotations to a custom dataset format similar to CPPE-5.

    Args:
        images_dir (str): Path to the images directory.
        labels_dir (str): Path to the YOLO labels directory.
        categories (list): List of category names.

    Returns:
        list: A dataset where each entry contains image metadata and associated objects.
    """
    dataset = []
    annotation_id = 0
    image_id = 0

    for image_file in sorted(os.listdir(images_dir)):
        if not image_file.endswith(('.jpg', '.png', '.jpeg')):
            continue

        image_path = os.path.join(images_dir, image_file)

        # Open the image as a PIL image object
        with Image.open(image_path) as img:
            width, height = img.size

            # Prepare the image entry
            image_entry = {
                'image_id': image_id,
                'image': img.copy(),  # Keep a reference to the PIL image
                'width': width,
                'height': height,
                'objects': {
                    'id': [],
                    'area': [],
                    'bbox': [],
                    'category': []
                }
            }

            # Corresponding label file in YOLO format
            label_file = os.path.join(labels_dir, image_file.rsplit('.', 1)[0] + '.txt')
            if os.path.exists(label_file):
                with open(label_file, 'r') as f:
                    for line in f:
                        parts = line.strip().split()
                        category_id = int(parts[0])
                        x_center, y_center, box_width, box_height = map(float, parts[1:])

                        # Convert YOLO to bounding box coordinates
                        x_min = (x_center - box_width / 2) * width
                        y_min = (y_center - box_height / 2) * height
                        bbox_width = box_width * width
                        bbox_height = box_height * height
                        area = bbox_width * bbox_height

                        # Append object data
                        image_entry['objects']['id'].append(annotation_id)
                        image_entry['objects']['area'].append(int(area))
                        image_entry['objects']['bbox'].append([
                            round(x_min, 1),
                            round(y_min, 1),
                            round(bbox_width, 1),
                            round(bbox_height, 1)
                        ])
                        image_entry['objects']['category'].append(category_id)

                        annotation_id += 1

            dataset.append(image_entry)
            image_id += 1

    return dataset


categories_kugellager = [
    {"id": 0, "name": "defect"},
]
images_dir_kugellager_train = os.path.join(BASE_DIR, "Data/Kugellager_Data/YOLO_Data/train/images")
labels_dir_kugellager_train = os.path.join(BASE_DIR, "Data/Kugellager_Data/YOLO_Data/train/labels")

dataset_kugellager_train = Dataset.from_list(convert_yolo_to_custom_format(images_dir_kugellager_train, labels_dir_kugellager_train, categories_kugellager))

images_dir_kugellager_test = os.path.join(BASE_DIR, "Data/Kugellager_Data/YOLO_Data/test/images")
labels_dir_kugellager_test = os.path.join(BASE_DIR, "Data/Kugellager_Data/YOLO_Data/test/labels")

dataset_kugellager_test = Dataset.from_list(convert_yolo_to_custom_format(images_dir_kugellager_test, labels_dir_kugellager_test, categories_kugellager))



images_dir_kugellager_train_halb = os.path.join(BASE_DIR, "Data/Kugellager_Data/YOLO_Data/train_halb/images")
labels_dir_kugellager_train_halb = os.path.join(BASE_DIR, "Data/Kugellager_Data/YOLO_Data/train_halb/labels")

dataset_kugellager_train_halb = Dataset.from_list(convert_yolo_to_custom_format(images_dir_kugellager_train_halb, labels_dir_kugellager_train_halb, categories_kugellager))

images_dir_kugellager_test_halb = os.path.join(BASE_DIR, "Data/Kugellager_Data/YOLO_Data/test_halb/images")
labels_dir_kugellager_test_halb = os.path.join(BASE_DIR, "Data/Kugellager_Data/YOLO_Data/test_halb/labels")

dataset_kugellager_test_halb = Dataset.from_list(convert_yolo_to_custom_format(images_dir_kugellager_test_halb, labels_dir_kugellager_test_halb, categories_kugellager))



categories_oberfläche = [
    {"id": 0, "name": "crazing"},
    {"id": 1, "name": "inclusion"},
    {"id": 2, "name": "patches"},
    {"id": 3, "name": "pitted surface"},
    {"id": 4, "name": "rolled in scale"},
    {"id": 5, "name": "scratches"}
]

images_dir_oberfläche_train = os.path.join(BASE_DIR, "Data/Oberflächen_Data/YOLO_Data/train/images")
labels_dir_oberfläche_train = os.path.join(BASE_DIR, "Data/Oberflächen_Data/YOLO_Data/train/labels")

dataset_oberfläche_train = Dataset.from_list(convert_yolo_to_custom_format(images_dir_oberfläche_train, labels_dir_oberfläche_train, categories_oberfläche))

images_dir_oberfläche_test = os.path.join(BASE_DIR, "Data/Oberflächen_Data/YOLO_Data/test/images")
labels_dir_oberfläche_test = os.path.join(BASE_DIR, "Data/Oberflächen_Data/YOLO_Data/test/labels")

dataset_oberfläche_test = Dataset.from_list(convert_yolo_to_custom_format(images_dir_oberfläche_test, labels_dir_oberfläche_test, categories_oberfläche))



images_dir_oberfläche_train_halb = os.path.join(BASE_DIR, "Data/Oberflächen_Data/YOLO_Data/train_halb/images")
labels_dir_oberfläche_train_halb = os.path.join(BASE_DIR, "Data/Oberflächen_Data/YOLO_Data/train_halb/labels")

dataset_oberfläche_train_halb = Dataset.from_list(convert_yolo_to_custom_format(images_dir_oberfläche_train_halb, labels_dir_oberfläche_train_halb, categories_oberfläche))

images_dir_oberfläche_test_halb = os.path.join(BASE_DIR, "Data/Oberflächen_Data/YOLO_Data/test_halb/images")
labels_dir_oberfläche_test_halb = os.path.join(BASE_DIR, "Data/Oberflächen_Data/YOLO_Data/test_halb/labels")

dataset_oberfläche_test_halb = Dataset.from_list(convert_yolo_to_custom_format(images_dir_oberfläche_test_halb, labels_dir_oberfläche_test_halb, categories_oberfläche))


In [6]:
print("CUDA available:", torch.cuda.is_available())
print("CUDA device name:", torch.cuda.get_device_name(0))
print("CUDA version:", torch.version.cuda)
print("cuDNN version:", torch.backends.cudnn.version())
total_memory = torch.cuda.get_device_properties(0).total_memory

# Belegter Speicher (in Bytes)
allocated_memory = torch.cuda.memory_allocated(0)

# Zwischengespeicherter Speicher
cached_memory = torch.cuda.memory_reserved(0)

print(f"Gesamtspeicher: {total_memory / 1e9:.2f} GB")
print(f"Belegter Speicher: {allocated_memory / 1e9:.2f} GB")
print(f"Zwischengespeicherter Speicher: {cached_memory / 1e9:.2f} GB")

CUDA available: True
CUDA device name: NVIDIA GeForce RTX 3060 Ti
CUDA version: 12.4
cuDNN version: 90100
Gesamtspeicher: 8.59 GB
Belegter Speicher: 0.00 GB
Zwischengespeicherter Speicher: 0.00 GB


In [7]:
def formatted_anns(image_id, category, area, bbox):
    annotations = []
    for i in range(0, len(category)):

        new_ann = {
            "id": image_id,
            "category_id": category[i],  # Hier wird das richtige category ID verwendet
            "isCrowd": 0,
            "area": area[i],
            "bbox": list(bbox[i]),
        }

        annotations.append(new_ann)
    return annotations

# Create annotations such that they match the expected form by the algorithm
def transform_ann(examples, image_processor = YolosImageProcessor()):
    image_ids = examples["image_id"]
    images, bboxes, area, categories = [], [], [], []
    for image, objects in zip(examples["image"], examples["objects"]): 
        image = np.array(image.convert("RGB"))[:, :, ::-1]
        area.append(objects["area"])
        images.append(image)
        bboxes.append(objects["bbox"])
        categories.append(objects["category"])

    targets = [
    {"image_id": id_, "annotations": formatted_anns(id_, cat_, ar_, box_)}
    for id_, cat_, ar_, box_ in zip(image_ids, categories, area, bboxes)
    ]

    return image_processor(images=images, annotations=targets, return_tensors="pt") # Is applied on the whole batch

def collate_fn(batch, image_processor = YolosImageProcessor()):
    pixel_values = [item["pixel_values"] for item in batch]
    encoding = image_processor.pad(pixel_values, return_tensors="pt")
    labels = [item["labels"] for item in batch]
    batch = {}
    batch["pixel_values"] = encoding["pixel_values"]
    #batch["pixel_mask"] = encoding["pixel_mask"] # For object detection we do not need this - only needed for segmentation.
    batch["labels"] = labels
    return batch


def model_training(categories, model_name, train_data, validation_data, num_epochs=3, image_processor=YolosImageProcessor(), output_name="Kugellager"):
    # Mapping zwischen IDs und Labels
    id2label = {category['id']: category['name'] for category in categories}
    label2id = {category['name']: category['id'] for category in categories} 

    # Trainingsargumente definieren
    training_args = TrainingArguments(

        output_dir=f"trained_model/{output_name}/{model_name}", 
        remove_unused_columns=False, 
        load_best_model_at_end=False, 
        save_strategy="no", 
        eval_strategy="epoch", 
        per_device_train_batch_size=10, 
        push_to_hub=False,
        logging_steps=10,
        num_train_epochs=num_epochs
    )

    # Modell initialisieren
    model = AutoModelForObjectDetection.from_pretrained(
        model_name,
        id2label=id2label,
        label2id=label2id,
        ignore_mismatched_sizes=True,  
    )

    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")  
    model.to(device)
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=collate_fn, 
        train_dataset=train_data,
        eval_dataset=validation_data,
        tokenizer=image_processor,
    )

    # Training durchführen
    trainer.train()

    # Evaluation durchführen
    evaluation_results = trainer.evaluate()

    # Plotten der Losses aus dem TrainerState
    train_logs = trainer.state.log_history

    # Extrahiere Training und Evaluation Losses
    train_losses = [log['loss'] for log in train_logs if 'loss' in log]
    eval_losses = [log['eval_loss'] for log in train_logs if 'eval_loss' in log]

    plt.figure(figsize=(14, 10))
    
    # Plot Training Loss
    plt.plot(train_losses, label='Training Loss', color='blue')
    
    # Plot Evaluation Loss
    # Stelle sicher, dass die Längen übereinstimmen
    if len(eval_losses) > 0:
        # Falls unterschiedliche Längen, interpoliere oder schneide
        if len(eval_losses) != len(train_losses):
            # Interpoliere eval_losses auf die Länge von train_losses
            x_train = np.linspace(0, len(train_losses)-1, len(train_losses))
            x_eval = np.linspace(0, len(train_losses)-1, len(eval_losses))
            eval_losses_interpolated = np.interp(x_train, x_eval, eval_losses)
            
            plt.plot(eval_losses_interpolated, label='Evaluation Loss', color='red')
        else:
            plt.plot(eval_losses, label='Evaluation Loss', color='red')

    plt.title(f'Loss Progression for {model_name}')
    plt.xlabel('Training Steps')
    plt.ylabel('Loss')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"trained_model/{output_name}/{model_name}/loss_plot.png")
    plt.close()

    save_path = f"trained_model/{output_name}/{model_name}/final_model"
    model.save_pretrained(save_path)

    return evaluation_results, train_losses, eval_losses
    

# Transform data such that it can be feed to the model
train_data_kugellager = dataset_kugellager_train.with_transform(transform_ann)
test_data_kugellager = dataset_kugellager_test.with_transform(transform_ann)

train_data_oberfläche = dataset_oberfläche_train.with_transform(transform_ann)
test_data_oberfläche = dataset_oberfläche_test.with_transform(transform_ann)


train_data_kugellager_halb = dataset_kugellager_train_halb.with_transform(transform_ann)
test_data_kugellager_halb = dataset_kugellager_test_halb.with_transform(transform_ann)

train_data_oberfläche_halb = dataset_oberfläche_train_halb.with_transform(transform_ann)
test_data_oberfläche_halb = dataset_oberfläche_test_halb.with_transform(transform_ann)

best_models_kugellager_dict = {}
best_models_oberfläche_dict = {}

best_models_kugellager_dict_halb = {}
best_models_oberfläche_dict_halb = {}

# "ultralytics/yolov11-m", 
# "jparedesDS/welding-defects-detection",
# "facebook/detr-resnet-50", 
# "hustvl/yolos-small",
# "hustvl/yolos-tiny"
# "jparedesDS/welding-defects-detection",
# "hustvl/yolos-small",

model_training_list = ["hustvl/yolos-tiny", "facebook/detr-resnet-50", ]

for model_name in model_training_list:
    evaluation_results, train_losses, eval_losses = model_training(
        categories=categories_kugellager, 
        model_name=model_name, 
        train_data=train_data_kugellager, 
        validation_data=test_data_kugellager, 
        num_epochs=50,
        output_name="Kugellager"
    )
    best_models_kugellager_dict[model_name] = {
        'evaluation_results': evaluation_results,
        'train_losses': train_losses,
        'eval_losses': eval_losses
    }


for model_name in model_training_list:
    evaluation_results, train_losses, eval_losses = model_training(
        categories=categories_oberfläche, 
        model_name=model_name, 
        train_data=train_data_oberfläche, 
        validation_data=test_data_oberfläche, 
        num_epochs=10,
        output_name="Oberfläche"
    )
    best_models_oberfläche_dict[model_name] = {
        'evaluation_results': evaluation_results,
        'train_losses': train_losses,
        'eval_losses': eval_losses
    }


for model_name in model_training_list:
    evaluation_results, train_losses, eval_losses = model_training(
        categories=categories_kugellager, 
        model_name=model_name, 
        train_data=train_data_kugellager_halb, 
        validation_data=test_data_kugellager_halb, 
        num_epochs=50,
        output_name="Kugellager_halb"
    )
    best_models_kugellager_dict_halb[model_name] = {
        'evaluation_results': evaluation_results,
        'train_losses': train_losses,
        'eval_losses': eval_losses
    }


for model_name in model_training_list:
    evaluation_results, train_losses, eval_losses = model_training(
        categories=categories_oberfläche, 
        model_name=model_name, 
        train_data=train_data_oberfläche_halb, 
        validation_data=test_data_oberfläche_halb, 
        num_epochs=20,
        output_name="Oberfläche_halb"
    )
    best_models_oberfläche_dict_halb[model_name] = {
        'evaluation_results': evaluation_results,
        'train_losses': train_losses,
        'eval_losses': eval_losses
    }

Some weights of YolosForObjectDetection were not initialized from the model checkpoint at hustvl/yolos-tiny and are newly initialized because the shapes did not match:
- class_labels_classifier.layers.2.bias: found shape torch.Size([92]) in the checkpoint and torch.Size([2]) in the model instantiated
- class_labels_classifier.layers.2.weight: found shape torch.Size([92, 192]) in the checkpoint and torch.Size([2, 192]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\anohl\AppData\Local\Temp\ipykernel_1852\1018351259.py:75: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Using device: cuda:0


  0%|          | 10/2400 [00:12<45:39,  1.15s/it] 

{'loss': 1.5067, 'grad_norm': 88.63176727294922, 'learning_rate': 4.979166666666667e-05, 'epoch': 0.21}


  1%|          | 20/2400 [00:23<44:30,  1.12s/it]

...

{'loss': 2.09, 'grad_norm': 102.3625259399414, 'learning_rate': 0.0, 'epoch': 20.0}



100%|██████████| 300/300 [05:27<00:00,  1.09s/it]


{'eval_loss': 3.1181180477142334, 'eval_runtime': 3.6755, 'eval_samples_per_second': 16.324, 'eval_steps_per_second': 2.177, 'epoch': 20.0}
{'train_runtime': 327.1202, 'train_samples_per_second': 9.171, 'train_steps_per_second': 0.917, 'train_loss': 3.1260866355895995, 'epoch': 20.0}


 38%|███▊      | 3/8 [00:00<00:01,  2.96it/s]

KeyboardInterrupt: 

In [7]:
for model_name, score in best_models_kugellager_dict.items():
    print(f"model: {model_name}, performte mit folgenden Scores:\n{score}")

for model_name, score in best_models_oberfläche_dict.items():
    print(f"model: {model_name}, performte mit folgenden Scores:\n{score}")

for model_name, score in best_models_kugellager_dict_halb.items():
    print(f"model: {model_name}, performte mit folgenden Scores:\n{score}")

for model_name, score in best_models_oberfläche_dict_halb.items():
    print(f"model: {model_name}, performte mit folgenden Scores:\n{score}")

model: hustvl/yolos-tiny, performte mit folgenden Scores:
{'evaluation_results': {'eval_loss': 0.9846168160438538, 'eval_runtime': 6.1671, 'eval_samples_per_second': 19.458, 'eval_steps_per_second': 2.432, 'epoch': 50.0}, 'train_losses': [1.5652, 1.5312, 1.2966, 1.3194, 1.2127, 1.2189, 1.3403, 1.3228, 1.3172, 1.6135, 1.1993, 1.1112, 1.2941, 1.2427, 1.2233, 1.1958, 1.0332, 1.1606, 1.1071, 1.1547, 1.0124, 1.0647, 1.0306, 1.0422, 1.1228, 0.9944, 1.1028, 1.0793, 1.0689, 1.0131, 1.0513, 0.9871, 1.0195, 0.9839, 1.0762, 1.1371, 1.0283, 1.1042, 1.0052, 0.8853, 0.9762, 0.9569, 0.9352, 0.9637, 0.8683, 0.8804, 0.8388, 0.9413, 1.0316, 0.8786, 0.8046, 0.8878, 0.8308, 0.7955, 0.8837, 0.9218, 0.8276, 0.7941, 0.7414, 0.7645, 0.9002, 0.8929, 0.8705, 0.8282, 0.8039, 0.8647, 0.7541, 0.8, 0.7175, 0.7535, 0.713, 0.8802, 0.6926, 0.7339, 0.7084, 0.6583, 0.7666, 0.711, 0.6861, 0.6481, 0.697, 0.7283, 0.6562, 0.7198, 0.6114, 0.6634, 0.6836, 0.6274, 0.7584, 0.8059, 0.7435, 0.6525, 0.7667, 0.6837, 0.6471, 0.6742,

In [ ]:

def load_model(model_path):
    """
    Lädt ein Object Detection Modell aus dem angegebenen Ordner.
    """
    model = AutoModelForObjectDetection.from_pretrained(model_path)
    processor = YolosImageProcessor.from_pretrained(model_path)
    model.eval()
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    model.to(device)
    return model, processor, device


def load_random_images(image_folder, num_images=10):
    """
    Lädt zufällig ausgewählte Bilder aus einem Ordner.
    """
    image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('jpg', 'jpeg', 'png'))]
    selected_files = random.sample(image_files, min(len(image_files), num_images))
    return [os.path.join(image_folder, f) for f in selected_files]


def draw_bounding_boxes(image, detections, labels):
    """
    Zeichnet Bounding Boxes und Labels auf das Bild.
    """
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()
    for detection in detections:
        box = detection['box']
        label = labels[detection['label']]
        score = detection['score']
        draw.rectangle(box, outline='red', width=3)
        draw.text((box[0], box[1] - 10), f"{label} {score:.2f}", fill="red", font=font)
    return image


def predict_and_label_images(model_path, image_folder, output_folder, num_images=10):
    """
    Lädt ein Modell, führt Object Detection auf zufälligen Bildern durch und speichert die Ergebnisse.
    """
    model, processor, device = load_model(model_path)
    images = load_random_images(image_folder, num_images)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for image_path in images:
        image = Image.open(image_path).convert("RGB")
        inputs = processor(images=image, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = model(**inputs)

        detections = []
        for logits, box in zip(outputs.logits[0], outputs.pred_boxes[0]):
            prob = logits.softmax(-1).max().item()
            label = logits.argmax().item()
            if prob > 0.5:  # Threshold
                detections.append({
                    'label': label,
                    'score': prob,
                    'box': [
                        box[0].item() * image.width, box[1].item() * image.height,
                        box[2].item() * image.width, box[3].item() * image.height
                    ]
                })

        labeled_image = draw_bounding_boxes(image, detections, model.config.id2label)
        output_path = os.path.join(output_folder, os.path.basename(image_path))
        labeled_image.save(output_path)


# Beispiel zur Nutzung:
model_path_oberfläche_yolo = os.path.join(BASE_DIR, "trained_model", "Oberfläche", "hustlv", "yolos-tiny", "final_model")
model_path_oberfläche_resnet = os.path.join(BASE_DIR, "trained_model", "Oberfläche", "facebook", "detr-resnet-50", "final_model")
model_path_kugellager_yolo = os.path.join(BASE_DIR, "trained_model", "Kugellager", "hustlv", "yolos-tiny", "final_model")
model_path_kugellager_resenet = os.path.join(BASE_DIR, "trained_model", "Kugellager", "facebook", "detr-resnet-50", "final_model")
predict_and_label_images(model_path=model_path_oberfläche_yolo, image_folder=images_dir_oberfläche_test, output_folder="test_labeling_pics_oberfläche_yolo", num_images=10)
predict_and_label_images(model_path=model_path_oberfläche_resnet, image_folder=images_dir_oberfläche_test, output_folder="test_labeling_pics_oberfläche_resnet", num_images=10)
predict_and_label_images(model_path=model_path_kugellager_yolo, image_folder=images_dir_oberfläche_test, output_folder="test_labeling_pics_kugellager_yolo", num_images=10)
predict_and_label_images(model_path=model_path_kugellager_resenet, image_folder=images_dir_oberfläche_test, output_folder="test_labeling_pics_kugellager_resnet", num_images=10)

In [8]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.synchronize()  # Stellt sicher, dass alle CUDA-Operationen abgeschlossen sind

# Optional: Garbage Collection erzwingen
import gc
gc.collect()

14063

In [9]:
# from transformers import AutoImageProcessor, pipeline
# from PIL import Image, ImageDraw, ImageFont

# random_images = random.sample(dataset_oberfläche_test["image"], 10)
# for test_image in random_images:
#     test = r"C:\Users\anohl\OneDrive\Dokumente\A_Uni_stuff\Albstadt\Semester 2\Computer_vision\Aufgaben\trained_model\hustvl\yolos-tiny\final_model"
#     out = pipeline("object-detection", model=test, image_processor=YolosImageProcessor())(test_image, threshold=0)

#     # Normalize the prediction thresholds
#     outs = [o["score"] for o in out]
#     filtered_out = []
#     threshold = 0.9
#     for o in out:
#         o["score"] = 1 / max(outs) * o["score"]
#         if o["score"] >= threshold:
#             filtered_out.append(o)
#             print(filtered_out)

#     def plot_box(img, results):
#         try:
#             image = Image.open(img)
#         except:
#             image = img
#         draw = ImageDraw.Draw(image)
#         for i, obj in enumerate(results):
#             box = [round(obj["box"][value], 2) for value in obj["box"].keys()]
#             x, y, x2, y2 = tuple(box)
#             draw.rectangle((x, y, x2, y2), outline="orange", width=2)
#             draw.text((x, y), f"{obj['label']} - {np.round(obj['score'], 3)}", fill="orange", stroke_width=2, stroke_fill="white")
#         image.save(f"./{np.random.randint(10e6)}.jpg")

#     # Visualize Results
#     plot_box(test_image, filtered_out)

In [10]:
# device = "cuda:0" if torch.cuda.is_available() else "cpu"
# print(f"Using device: {device}")

# # Verwendung von yolv8m um eine Blance zwischen performance und Genauigkeit zu haben 
# model = YOLO("yolov8m.pt")

# # Train the model
# path_to_yolo_yaml = os.path.join(BASE_DIR, "yaml_files", "yolo_dataset.yaml")
# train_results = model.train(
#     data=path_to_yolo_yaml,
#     epochs=400,
#     imgsz=150, # Weil das die tatsächliche Größe unserer Bilder darstellt
#     device=device,
#     batch=16,
#     mosaic=1.0,
# )

# # Evaluate model performance on the validation set
# metrics = model.val()
# print(metrics)

# # Export the model to the same directory as the script
# export_path = os.path.join(BASE_DIR, "Models", "yolo_kugellager_modell.pt")
# model_path = model.export()
# shutil.move(src=model_path, dst=export_path)
# print(f"Model exported to: {export_path}")


In [11]:
# metrics = model.val()
# print(f"Precision: {metrics['precision']}")
# print(f"Recall: {metrics['recall']}")
# print(f"mAP@0.5: {metrics['map50']}")
# print(f"mAP@0.5:0.95: {metrics['map']}")